In [1]:
import pandas as pd
import re
import emoji
# from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
# from matplotlib import pyplot as plt
# import numpy as np
# import datetime

In [2]:
df_tweets = pd.read_csv("data/tab2/aggregated_twitter_data.csv")

In [3]:
df_tweets

,brand,date,language,content,sentiment_neg,sentiment_neu,sentiment_pos
0,Airbnb,2019-07-31 23:54:53+00:00,en,im going on early tonight!! im at an #airbnb a...,"0,0","0,629","0,371"
1,Airbnb,2020-11-12 11:05:43+00:00,en,Join GOGLA as we are broadening the conversati...,"0,0","0,85","0,15"
2,Airbnb,2019-07-31 23:54:13+00:00,de,Petite Gem in Muir Beach CAhttps://www.airbnb....,NaN,NaN,NaN
3,Airbnb,2019-07-31 23:41:49+00:00,et,#ゲストハウス #ホステル #京橋 #大阪城\n#大阪 #日本 #バックパッカー #一人旅 ...,NaN,NaN,NaN
4,Airbnb,2019-07-31 23:15:04+00:00,en,Another day another #airbnb cancellation with ...,"0,059","0,838","0,103"
...,...,...,...,...,...,...,...
2737421,starbucks,2020-10-01 00:21:27+00:00,en,#Starbucks hikes dividend by 9.8 percent. $SBUX,"0,0","1,0","0,0"
2737422,starbucks,2020-10-01 00:13:44+00:00,ja,やっと飲めました🍠✨思ったよりしつこくなくて飲みやすかったです☺️💕\n10月ですね🍁\n#...,NaN,NaN,NaN
2737423,starbucks,2020-10-01 00:12:04+00:00,en,Mobile Friendly &amp; Responsive Design \n#mob...,"0,0","0,769","0,231"
2737424,starbucks,2020-10-01 00:11:56+00:00,en,can we all agree red can’t be sus when he look...,"0,0","0,691","0,309"


In [17]:
# Remove hours and minutes from the date
df_tweets['date'] = df_tweets['date'].str[:10]
# df_tweets['date'] = pd.to_datetime(df_tweets['date']).dt.date
df_tweets = df_tweets[df_tweets["date"].between("2019-06-31", "2020-10-31", inclusive = "both")]

In [20]:
df_tweets.rename(columns={'language': 'language_code'}, inplace=True)
query_string_lang_codes = "https://raw.githubusercontent.com/datasets/language-codes/master/data/language-codes.csv"
df_lang_codes = pd.read_csv(query_string_lang_codes)
df_lang_codes.rename(columns={'English': 'language',"alpha2":"language_code"}, inplace=True)
df_lang_codes

,language_code,language
0,aa,Afar
1,ab,Abkhazian
2,ae,Avestan
3,af,Afrikaans
4,ak,Akan
...,...,...
179,yi,Yiddish
180,yo,Yoruba
181,za,Zhuang; Chuang
182,zh,Chinese


In [21]:
# Join the language codes with the tweets
df_tweets = df_tweets.merge(df_lang_codes, on='language_code', how='left')
# Drop NaN values in the language_code column
df_tweets = df_tweets.dropna(subset=['language_code'])

In [22]:
df_tweets[df_tweets['language'].isnull()]

,brand,date,language_code,content,sentiment_neg,sentiment_neu,sentiment_pos,language
5,Airbnb,2019-07-31,zh-cn,补贴点家用容易嘛…还要跟客房阿姨一样收拾卫生:) #airbnb https://t.co/...,NaN,NaN,NaN,NaN
18001,Airbnb,2019-08-09,zh-cn,#中国，#长城，#过夜，#星星，#作伴，#Airbnb，我要去长城过夜 星星与我作伴 - r...,NaN,NaN,NaN,NaN
26453,Airbnb,2019-09-15,zh-cn,#Airbnb 你们他🐎是被警察抓走了吗？饮料不喝为什么要买？桃子不吃切你🐎痹 https:...,NaN,NaN,NaN,NaN
27694,Airbnb,2019-09-11,zh-tw,來到胡志明市，第一時間來個Room Tour先，33/F 的Airbnb，2房時尚靚裝，座擁...,NaN,NaN,NaN,NaN
29590,Airbnb,2019-09-04,zh-tw,假使Airbnb 公司制定了一個立意良善、非常好的政策，基於同事間的互助與感情的維繫，當有員...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2690887,starbucks,2020-07-21,zh-tw,今天收到的禮物，讓我的一天有個美好的開始~ #Starbucks https://t.co/...,NaN,NaN,NaN,NaN
2692637,starbucks,2020-07-16,zh-cn,第一次体验在东京国立市开业的全球第五家“手语星巴克”，店内的共同语言为手语，大部分店员都是聋...,NaN,NaN,NaN,NaN
2694670,starbucks,2020-07-10,zh-cn,星巴克超迷你2樓店，下午人塞滿滿，買了在公園喝好像老人🤣\n\n#starbucks #玩味...,NaN,NaN,NaN,NaN
2715040,starbucks,2020-09-20,zh-tw,晨晨在練鋼琴，特別叮囑我不能吃她的爆米花🍿️⋯⋯\n\n#STARBUCKS @ 莊爺爺の家...,NaN,NaN,NaN,NaN


In [23]:
# Set the correct language for the tweets with NaN values
df_tweets.loc[df_tweets['language'].isnull(), 'language'] = 'Chinese'

In [24]:
# Delete content for all tweets that are not in English
df_tweets.loc[df_tweets['language'] != 'English', 'content'] = ''


In [25]:
# Function to clean tweets from emojis and links and line breaks and more 
# Source https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/1602c98bb875368fa8edf18a372fbd92419c57a2/Dash_More_Advanced_Shit/NLP/nlpDash.py#L14
def clean_tweets(txt):
	txt = re.sub(r"RT[\s]+", "", txt)
	txt = txt.replace("\n", " ")
	txt = re.sub(" +", " ", txt)
	txt = re.sub(r"https?:\/\/\S+", "", txt)
	txt = re.sub(r"(@[A-Za-z0–9_]+)|[^\w\s]|#", "", txt)
	txt = emoji.replace_emoji(txt, replace='')
	txt.strip()
	return txt

In [31]:
df_tweets.dropna(inplace=True)
df_tweets['content'] = df_tweets['content'].map(lambda x: clean_tweets(x))

In [7]:
# replace , with .
df_tweets["sentiment_neg"] = df_tweets["sentiment_neg"].str.replace(",", ".").astype(float)
df_tweets["sentiment_pos"] = df_tweets["sentiment_pos"].str.replace(",", ".").astype(float)
df_tweets["sentiment_neu"] = df_tweets["sentiment_neu"].str.replace(",", ".").astype(float)

In [28]:
df_tweets['brand'] = df_tweets['brand'].str.title()
df_tweets["brand"].replace({"Cocacola":"Coca-Cola","Hm" : "HM"}, inplace=True)

In [34]:
# Save the preprocessed tweets to a csv file
df_tweets.to_csv("data/tab2/preprocessed_tweets.csv", index=False)

In [35]:
df_tweets_no_content = df_tweets.drop(columns=['content'])
df_tweets_no_content = df_tweets_no_content.drop(columns=['language_code'])

In [38]:
df_tweets_no_content.to_csv("data/tab2/preprocessed_tweets_no_content.csv", index=False)

In [2]:
df_tweets_no_content = pd.read_csv("data/tab2/preprocessed_tweets_no_content.csv")

In [42]:
# Replace Spanish; Castilian  with Spanish
df_tweets_no_content["language"] = df_tweets_no_content["language"].replace("Spanish; Castilian", "Spanish")

### Calculate Sentiment over time

In [83]:
df_sentiment_overall = df_tweets_no_content[["brand","sentiment_neg","sentiment_pos"]].groupby("brand").mean().reset_index()
df_sentiment_overall["sentiment_neg_perc"] = ((df_sentiment_overall["sentiment_neg"] / (df_sentiment_overall["sentiment_neg"] + df_sentiment_overall["sentiment_pos"]))*100).round(2)
df_sentiment_overall["sentiment_pos_perc"] = ((df_sentiment_overall["sentiment_pos"] / (df_sentiment_overall["sentiment_neg"] + df_sentiment_overall["sentiment_pos"]))*100).round(2)
df_sentiment_overall.to_csv("data/tab2/twitter_sentiment_overall.csv", index=False)
df_sentiment_overall

,brand,sentiment_neg,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc
0,Airbnb,0.031624,0.114082,21.70,78.30
1,Apple,0.029379,0.096634,23.31,76.69
2,Coca-Cola,0.029607,0.106577,21.74,78.26
3,HM,0.010769,0.241806,4.26,95.74
4,Nike,0.021891,0.147280,12.94,87.06
5,Starbucks,0.033937,0.139304,19.59,80.41


In [18]:
df_sentiment = df_tweets_no_content.groupby(["brand", "date"]).mean().reset_index()
df_sentiment

,brand,date,sentiment_neg,sentiment_neu,sentiment_pos
0,Airbnb,2019-07-01,0.026629,0.884532,0.088835
1,Airbnb,2019-07-02,0.024659,0.873564,0.101766
2,Airbnb,2019-07-03,0.034013,0.855670,0.110309
3,Airbnb,2019-07-04,0.029145,0.841529,0.129322
4,Airbnb,2019-07-05,0.022983,0.856261,0.120767
...,...,...,...,...,...
2921,Starbucks,2020-10-26,0.030316,0.817961,0.151727
2922,Starbucks,2020-10-27,0.018603,0.865257,0.116140
2923,Starbucks,2020-10-28,0.020458,0.860097,0.119444
2924,Starbucks,2020-10-29,0.027386,0.840475,0.132143


In [85]:
df_sentiment_day = df_sentiment
df_sentiment_day["sentiment_neg_perc"] = ((df_sentiment_day["sentiment_neg"] / (df_sentiment_day["sentiment_neg"] + df_sentiment_day["sentiment_pos"]))*100).round(2)
df_sentiment_day["sentiment_pos_perc"] = ((df_sentiment_day["sentiment_pos"] / (df_sentiment_day["sentiment_neg"] + df_sentiment_day["sentiment_pos"]))*100).round(2)
df_sentiment_day["polarity"] = df_sentiment_day["sentiment_pos_perc"] - df_sentiment_day["sentiment_neg_perc"]
df_sentiment_day.rename(columns={"date":"Date"}, inplace=True)
df_sentiment_day.to_csv("data/tab2/twitter_sentiment_daily_percent.csv", index=False)

In [86]:
df_sentiment_day

,brand,Date,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc,polarity
0,Airbnb,2019-07-01,0.026629,0.884532,0.088835,23.06,76.94,53.88
1,Airbnb,2019-07-02,0.024659,0.873564,0.101766,19.51,80.49,60.98
2,Airbnb,2019-07-03,0.034013,0.855670,0.110309,23.57,76.43,52.86
3,Airbnb,2019-07-04,0.029145,0.841529,0.129322,18.39,81.61,63.22
4,Airbnb,2019-07-05,0.022983,0.856261,0.120767,15.99,84.01,68.02
...,...,...,...,...,...,...,...,...
2921,Starbucks,2020-10-26,0.030316,0.817961,0.151727,16.65,83.35,66.70
2922,Starbucks,2020-10-27,0.018603,0.865257,0.116140,13.81,86.19,72.38
2923,Starbucks,2020-10-28,0.020458,0.860097,0.119444,14.62,85.38,70.76
2924,Starbucks,2020-10-29,0.027386,0.840475,0.132143,17.17,82.83,65.66


### Weeks

In [3]:
df_stocks_weeks = pd.read_csv("data/tab2/aggregated_stocks_values_weeks.csv")

In [29]:
# Get the year-week from the date column
df_sentiment["date"] = pd.to_datetime(df_sentiment["date"])
df_sentiment["week"] = df_sentiment["date"].dt.isocalendar().week
df_sentiment["year"] = df_sentiment["date"].dt.year
df_sentiment["week_year"] = df_sentiment["year"].astype(str) + "-" + df_sentiment["week"].astype(str)
df_sentiment["week_year"].replace({"2019-1": "2020-1"}, inplace=True)

In [41]:
df_stocks_weeks["Date_64"] = pd.to_datetime(df_stocks_weeks["Date"])
df_stocks_weeks["week"] = df_stocks_weeks["Date_64"].dt.isocalendar().week
df_stocks_weeks["year"] = df_stocks_weeks["Date_64"].dt.year
df_stocks_weeks["week_year"] = df_stocks_weeks["year"].astype(str) + "-" + df_stocks_weeks["week"].astype(str)
df_stocks_weeks["week_year"].replace({"2019-1": "2020-1"}, inplace=True)

In [44]:
df_sentiment["date_week"] = df_sentiment["week_year"].replace(df_stocks_weeks["week_year"].values, df_stocks_weeks["Date"].values)
df_sentiment_weekly = df_sentiment.groupby(["brand", "date_week"]).mean().reset_index()
df_sentiment_weekly["sentiment_neg_perc"] = ((df_sentiment_weekly["sentiment_neg"] / (df_sentiment_weekly["sentiment_neg"] + df_sentiment_weekly["sentiment_pos"]))*100).round(2)
df_sentiment_weekly["sentiment_pos_perc"] = ((df_sentiment_weekly["sentiment_pos"] / (df_sentiment_weekly["sentiment_neg"] + df_sentiment_weekly["sentiment_pos"]))*100).round(2)
df_sentiment_weekly["polarity"] = df_sentiment_weekly["sentiment_pos_perc"] - df_sentiment_weekly["sentiment_neg_perc"]
df_sentiment_weekly.rename(columns={"date_week":"Date"}, inplace=True)

In [5]:
# Get week year from date
# df_sentiment["date"] = pd.to_datetime(df_sentiment["date"])
# df_sentiment["week_year"] = df_sentiment["date"].dt.strftime("%Y-%U")
# df_stocks_weeks["Date_64"] = df_stocks_weeks["Date"].astype("datetime64")
# df_stocks_weeks["week_year"] = df_stocks_weeks["Date_64"].dt.strftime("%Y-%U")

# df_stocks_weeks["week_year"].replace({"2021-00": "2020-52"}, inplace=True)
# df_sentiment["week_year"].replace({"2021-00": "2020-52"}, inplace=True)

In [8]:
# df_stocks_weeks

,Date,Open,High,Low,Close,Adj Close,Volume,Brand Abbreviation,pct_change,Brand Name,Date_64,week_year
0,2019-07-01,50.79,51.27,50.16,51.06,49.68,291262800,AAPL,NaN,Apple,2019-07-01,2019-26
1,2019-07-08,50.20,51.10,49.60,50.83,49.46,406402800,AAPL,-0.450450,Apple,2019-07-08,2019-27
2,2019-07-15,51.02,51.62,50.59,50.65,49.28,349566400,AAPL,-0.354122,Apple,2019-07-15,2019-28
3,2019-07-22,50.91,52.43,50.90,51.94,50.54,348612800,AAPL,2.546890,Apple,2019-07-22,2019-29
4,2019-07-29,52.12,55.34,50.41,51.01,49.63,879082000,AAPL,-1.790528,Apple,2019-07-29,2019-30
...,...,...,...,...,...,...,...,...,...,...,...,...
557,2021-02-22,200.70,214.10,175.00,206.35,206.35,46497200,ABNB,2.625951,Airbnb,2021-02-22,2021-08
558,2021-03-01,195.82,200.00,161.90,179.81,179.81,54318500,ABNB,-12.861643,Airbnb,2021-03-01,2021-09
559,2021-03-08,179.99,208.70,177.03,206.74,206.74,28801700,ABNB,14.976920,Airbnb,2021-03-08,2021-10
560,2021-03-15,209.54,215.49,187.81,194.39,194.39,20873000,ABNB,-5.973687,Airbnb,2021-03-15,2021-11


In [9]:
# Set every week_year from df_sentiment to be equal to the Date column from df_stocks_weeks
# df_sentiment["date_week"] = df_sentiment["week_year"].replace(df_stocks_weeks["week_year"].values, df_stocks_weeks["Date"].values)
# df_sentiment_weekly = df_sentiment.groupby(["brand", "date_week"]).mean().reset_index()
# df_sentiment_weekly["sentiment_neg_perc"] = ((df_sentiment_weekly["sentiment_neg"] / (df_sentiment_weekly["sentiment_neg"] + df_sentiment_weekly["sentiment_pos"]))*100).round(2)
# df_sentiment_weekly["sentiment_pos_perc"] = ((df_sentiment_weekly["sentiment_pos"] / (df_sentiment_weekly["sentiment_neg"] + df_sentiment_weekly["sentiment_pos"]))*100).round(2)
# df_sentiment_weekly["polarity"] = df_sentiment_weekly["sentiment_pos_perc"] - df_sentiment_weekly["sentiment_neg_perc"]
# df_sentiment_weekly.rename(columns={"date_week":"Date"}, inplace=True)

In [47]:
df_sentiment_weekly.to_csv("data/tab2/twitter_sentiment_weekly_percent.csv", index=False)

### Months

In [ ]:
df_sentiment["year_month"] = df_sentiment["date_axis"].dt.strftime("%Y-%m")
df_sentiment

In [ ]:
df_sentiment = df_sentiment.groupby(["brand", "year_month"]).mean().reset_index()
df_sentiment["sentiment_neg_perc"] = df_sentiment["sentiment_neg"] / (df_sentiment["sentiment_neg"] + df_sentiment["sentiment_pos"])
df_sentiment["sentiment_pos_perc"] = df_sentiment["sentiment_pos"] / (df_sentiment["sentiment_neg"] + df_sentiment["sentiment_pos"])
df_sentiment["polarity"] = df_sentiment["sentiment_pos_perc"] - df_sentiment["sentiment_neg_perc"]
df_sentiment

In [66]:
df_sentiment["sentiment_neg_perc"] = (df_sentiment["sentiment_neg_perc"]*100).round(2)
df_sentiment["sentiment_pos_perc"] = (df_sentiment["sentiment_pos_perc"]*100).round(2)
df_sentiment["polarity"] = (df_sentiment["polarity"]*100).round(2)
df_sentiment.rename(columns={"year_month":"date_axis"}, inplace=True)
df_sentiment

,brand,date_axis,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc,polarity
0,Airbnb,2019-07,0.025080,0.857850,0.117067,17.64,82.36,0.647125
1,Airbnb,2019-08,0.026106,0.853321,0.120572,17.80,82.20,0.644036
2,Airbnb,2019-09,0.023909,0.852364,0.123727,16.19,83.81,0.676107
3,Airbnb,2019-10,0.027352,0.854657,0.117992,18.82,81.18,0.623625
4,Airbnb,2019-11,0.043070,0.837852,0.119073,26.56,73.44,0.468739
...,...,...,...,...,...,...,...,...
91,Starbucks,2020-06,0.043283,0.828457,0.128259,25.23,74.77,0.495368
92,Starbucks,2020-07,0.034183,0.829184,0.136633,20.01,79.99,0.599770
93,Starbucks,2020-08,0.029399,0.842288,0.128312,18.64,81.36,0.627181
94,Starbucks,2020-09,0.031994,0.832585,0.135424,19.11,80.89,0.617791


In [98]:
# Add last day of the month to every date in date_axis
df_sentiment["date_axis"] = pd.to_datetime(df_sentiment["date_axis"])
df_sentiment["date_axis"] = df_sentiment["date_axis"] + pd.offsets.MonthEnd(0)
df_sentiment

,brand,date_axis,sentiment_neg,sentiment_neu,sentiment_pos,sentiment_neg_perc,sentiment_pos_perc
0,Airbnb,2019-07-31,0.020891,0.841844,0.137284,13.21,86.79
1,Airbnb,2019-08-31,0.026106,0.853321,0.120572,17.80,82.20
2,Airbnb,2019-09-30,0.023909,0.852364,0.123727,16.19,83.81
3,Airbnb,2019-10-31,0.027352,0.854657,0.117992,18.82,81.18
4,Airbnb,2019-11-30,0.043070,0.837852,0.119073,26.56,73.44
...,...,...,...,...,...,...,...
91,Starbucks,2020-06-30,0.043283,0.828457,0.128259,25.23,74.77
92,Starbucks,2020-07-31,0.034183,0.829184,0.136633,20.01,79.99
93,Starbucks,2020-08-31,0.029399,0.842288,0.128312,18.64,81.36
94,Starbucks,2020-09-30,0.031994,0.832585,0.135424,19.11,80.89


In [101]:
df_sentiment.to_csv("data/tab2/twitter_sentiment_monthly_percent.csv", index=False)

### Get tweets count

In [4]:
df_tweets_count_daily = df_tweets_no_content[["language","brand","date"]].groupby(["brand", "date"]).count().reset_index()
df_tweets_count_daily.rename(columns={"language":"tweets_count","date":"Date"}, inplace=True)
df_tweets_count_daily.to_csv("data/tab2/twitter_count_daily.csv", index=False)

In [5]:
# Get the year-week from the date column
df_tweets_count_daily["Date"] = pd.to_datetime(df_tweets_count_daily["Date"])
df_tweets_count_daily["week"] = df_tweets_count_daily["Date"].dt.isocalendar().week
df_tweets_count_daily["year"] = df_tweets_count_daily["Date"].dt.year
df_tweets_count_daily["week_year"] = df_tweets_count_daily["year"].astype(str) + "-" + df_tweets_count_daily["week"].astype(str)
df_tweets_count_daily["week_year"].replace({"2019-1": "2020-1"}, inplace=True)

In [9]:
df_tweets_count_daily[df_tweets_count_daily["week_year"] == "2019-52"]

,brand,Date,tweets_count,week,year,week_year
175,Airbnb,2019-12-23,194,52,2019,2019-52
176,Airbnb,2019-12-24,181,52,2019,2019-52
177,Airbnb,2019-12-25,146,52,2019,2019-52
178,Airbnb,2019-12-26,205,52,2019,2019-52
179,Airbnb,2019-12-27,305,52,2019,2019-52
180,Airbnb,2019-12-28,235,52,2019,2019-52
181,Airbnb,2019-12-29,245,52,2019,2019-52
661,Apple,2019-12-23,1819,52,2019,2019-52
662,Apple,2019-12-24,1562,52,2019,2019-52
663,Apple,2019-12-25,1326,52,2019,2019-52


In [6]:
df_stocks_weeks = pd.read_csv("data/tab2/aggregated_stocks_values_weeks.csv")
df_stocks_weeks["Date_64"] = pd.to_datetime(df_stocks_weeks["Date"])
df_stocks_weeks["week"] = df_stocks_weeks["Date_64"].dt.isocalendar().week
df_stocks_weeks["year"] = df_stocks_weeks["Date_64"].dt.year
df_stocks_weeks["week_year"] = df_stocks_weeks["year"].astype(str) + "-" + df_stocks_weeks["week"].astype(str)
df_stocks_weeks["week_year"].replace({"2019-1": "2020-1"}, inplace=True)

In [12]:
df_tweets_count_daily["date_week"] = df_tweets_count_daily["week_year"].replace(df_stocks_weeks["week_year"].values, df_stocks_weeks["Date"].values)
df_tweets_count_weekly = df_tweets_count_daily.groupby(["brand", "date_week"]).sum().reset_index()
df_tweets_count_weekly.rename(columns={"date_week":"Date"}, inplace=True)
df_tweets_count_weekly.drop(columns=["week","year"], inplace=True)
df_tweets_count_weekly.to_csv("data/tab2/twitter_count_weekly.csv", index=False)

In [15]:
df_tweets_count_weekly

,brand,Date,tweets_count
0,Airbnb,2019-07-01,2227
1,Airbnb,2019-07-08,2403
2,Airbnb,2019-07-15,2329
3,Airbnb,2019-07-22,2465
4,Airbnb,2019-07-29,2571
...,...,...,...
415,Starbucks,2020-09-28,3044
416,Starbucks,2020-10-05,2478
417,Starbucks,2020-10-12,2650
418,Starbucks,2020-10-19,2417


In [94]:
df_tweets_count_daily["year_month"] = df_tweets_count_daily["Date"].dt.strftime("%Y-%m")
df_tweets_count_monthly = df_tweets_count_daily.groupby(["brand", "year_month"]).sum().reset_index()
df_tweets_count_monthly.rename(columns={"year_month":"Date"}, inplace=True)

In [95]:
df_tweets_count_monthly.to_csv("data/tab2/twitter_count_monthly.csv", index=False)

### Calculate languages

In [36]:
# Set Estonian to Japanese as it was incorrectly detected
df_lang = df_tweets_no_content
df_lang["language"] = df_lang["language"].replace("Estonian", "Japanese")

In [37]:
df_lang = df_tweets_no_content[["brand","date","language"]].groupby(["brand", "language"]).count().reset_index()
df_lang.rename(columns={"date": "count"}, inplace=True)
df_lang

,brand,language,count
0,Airbnb,Afrikaans,168
1,Airbnb,Albanian,4
2,Airbnb,Arabic,216
3,Airbnb,Bengali,1
4,Airbnb,Bulgarian,50
...,...,...,...
305,Starbucks,Turkish,3540
306,Starbucks,Ukrainian,11
307,Starbucks,Urdu,8
308,Starbucks,Vietnamese,58


In [40]:
# For every brand, every language not in the top 5 is grouped into "other"
df_lang = df_lang.sort_values(by=["brand","count"], ascending=False).reset_index(drop=True)
df_lang["language"] = np.where(df_lang.groupby("brand")["count"].rank(method="first", ascending=False) > 5, "Others", df_lang["language"])
df_lang = df_lang.groupby(["brand", "language"]).sum().reset_index()
df_lang.to_csv("data/tab2/twitter_languages_count.csv", index=False)
df_lang

,brand,language,count
0,Airbnb,English,90037
1,Airbnb,French,8875
2,Airbnb,Japanese,9219
3,Airbnb,Others,15516
4,Airbnb,Spanish,9727
5,Apple,English,921456
6,Apple,French,64819
7,Apple,Japanese,109366
8,Apple,Others,204696
9,Apple,Spanish,76439
